In [1]:
import numpy as np
import pandas as pd


from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [2]:
data = pd.read_csv('C:/Users/aalmpani/PML/crowdflower-sentiment-analysis-in-text/data/text_emotion.csv')
# Keeping only the necessary columns
data = data[['content','sentiment']]

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence

data = data[data.sentiment != "neutral"]
data['content'] = data['content'].apply(lambda x: x.lower())
data['content'] = data['content'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


#for idx,row in data.iterrows():
 #   row[0] = row[0].replace('rt',' ')

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['content'].values)
tokens = tokenizer.word_index
X = tokenizer.texts_to_sequences(data['content'].values)
X = pad_sequences(X)
vocabulary_size = len(tokens) + 1

In [26]:
from keras.models import Sequential
model = Sequential()

model.add(Embedding(vocabulary_size, 10,input_length = X.shape[1]))
model.add(LSTM(units = 128))
model.add(LSTM(units = 64))
model.add(Dense(units=100, activation='relu'))
#model.add(Dropout(0.45)
model.add(Dense(units=5, activation='softmax'))

#Compile Model
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 32, 10)            442020    
                                                                 
 lstm_7 (LSTM)               (None, 128)               71168     
                                                                 
 dense (Dense)               (None, 5)                 645       
                                                                 
Total params: 513,833
Trainable params: 513,833
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 32, 10)            442020    
                                                                 
 lstm_7 (LSTM)               (None, 128)               71168     
                                                                 
 dense (Dense)               (None, 5)                 645       
                                                                 
Total params: 513,833
Trainable params: 513,833
Non-trainable params: 0
_________________________________________________________________


In [28]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(21012, 32) (21012, 12)
(10350, 32) (10350, 12)


In [29]:
#Train Model
model.fit(X_train, Y_train, epochs = 10, batch_size=256)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\aalmpani\Anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\aalmpani\Anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\aalmpani\Anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\aalmpani\Anaconda3\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\aalmpani\Anaconda3\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "C:\Users\aalmpani\Anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_wei